In [ ]:
# generate a random pair q0,v0
# solve for x_ipopt using ipopt
# we want loss = (x_lag - x_ipopt)^2 == 0
# compute the gradient of the loss with respect to the parameters
# update the parameters using SGD
# repeat

In [2]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds
using LCPSim

using RigidBodyTreeInspector
using DrakeVisualizer
using Plots

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/8c8a043c2f00369f3e7ae550277d2a54e59a34ad-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [23]:
urdf = joinpath("..","urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)

env = LCPSim.parse_contacts(mechanism, urdf);
Δt = 0.01;
world = root_body(mechanism)
world_frame = default_frame(world)
total_weight = mass(mechanism) * norm(mechanism.gravitational_acceleration)

num_q = num_positions(mechanism)
num_v = num_velocities(mechanism)
num_x = num_q + num_v
num_contacts = length(env.contacts)
bodies = []
contact_points = []
obstacles = []
contact_bases = []
μs = []
paths = []
Ds = []
for (body, contact_point, obstacle) in env.contacts
     push!(bodies, body)
     push!(contact_points, contact_point)
     push!(obstacles, obstacle)
     push!(contact_bases, contact_basis(obstacle))
     push!(μs, obstacle.μ)
     push!(paths, path(mechanism, body, world))
     push!(Ds, contact_basis(obstacle))
end
β_dim = length(contact_bases[1])
β_selector = find(repmat(vcat(ones(β_dim),[0,0]),num_contacts))
λ_selector = find(repmat(vcat(zeros(β_dim),[1,0]),num_contacts))
c_n_selector = find(repmat(vcat(zeros(β_dim),[0,1]),num_contacts))

rel_transforms = Vector{Tuple{Transform3D, Transform3D}}(num_contacts) # force transform, point transform
geo_jacobians = Vector{GeometricJacobian}(num_contacts)
x0 = MechanismState(mechanism)
xnext = MechanismState(mechanism)
ϕs = Vector{Float64}(num_contacts)
Dtv = Matrix{Float64}(β_dim,num_contacts);

num_var = num_contacts*(β_dim+2)

# var := [β,λ,c_n,β2,λ2,c_n2,...]    
var0 = zeros(Float64,num_var)

f = x̃ -> f_contact(x̃,ϕs,μs,Dtv,β_selector,λ_selector,c_n_selector,β_dim,num_contacts)
h = x̃ -> h_contact(x̃,HΔv,Δt,u0,dyn_bias,num_v,num_contacts,β_dim,bodies,contact_points,obstacles,Ds,
                   world_frame,total_weight,rel_transforms,geo_jacobians)
g = x̃ -> g_contact(x̃,num_contacts,β_dim,β_selector,λ_selector,c_n_selector,Dtv,μs)
    
num_h = num_v
num_g = num_contacts*(β_dim+2) + β_dim*num_contacts + num_contacts
    
# parameters of the augmented lagrangian method
num_steps = 5
α_vect0 = [1.^i for i in 1:num_steps]
c_vect0 = [2.^i for i in 1:num_steps]
I = eye(num_var);

In [24]:
q0 = [1., 0., 0., 0., 0., 0., 0.]
v0 = [0., 0., 0., 0., 0. , 0.]
u0 = zeros(num_v)
qnext = [1., 0., 0., 0., 0., 0., 0.]
vnext = [0., 0., 0., 0., 0., 0.]

set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
set_configuration!(xnext, qnext)
set_velocity!(xnext, vnext)
setdirty!(xnext)

H = mass_matrix(x0)
for i = 1:num_contacts
    v = point_velocity(twist_wrt_world(xnext,bodies[i]), transform_to_root(xnext, contact_points[i].frame) * contact_points[i])
    Dtv[:,i] = map(contact_bases[i]) do d
        dot(transform_to_root(xnext, d.frame) * d, v)
    end
    rel_transforms[i] = (relative_transform(xnext, obstacles[i].contact_face.outward_normal.frame, world_frame),
                         relative_transform(xnext, contact_points[i].frame, world_frame))
    geo_jacobians[i] = geometric_jacobian(xnext, paths[i])
            
    ϕs[i] = RigidBodyDynamics.separation(obstacles[i], transform(xnext, contact_points[i], obstacles[i].contact_face.outward_normal.frame))
end    
config_derivative = configuration_derivative(xnext)
HΔv = H * (vnext - v0)
dyn_bias = dynamics_bias(xnext);

In [25]:
# x = augmented_lagrangian_method(var0,f,h,g,num_h,num_g,α_vect0,c_vect0,I)
J = ForwardDiff.jacobian(z̃ -> augmented_lagrangian_method(var0,f,h,g,num_h,num_g,z̃[1:num_steps],z̃[num_steps+1:2*num_steps],I), vcat(α_vect0,c_vect0))
show(STDOUT, "text/plain", J); println("")

# x = ipopt_solve(x0,f,h,g,num_h,num_g)
# println(τ_total(x,num_v,num_contacts,β_dim,bodies,contact_points,obstacles,Ds,
#                world_frame,total_weight,rel_transforms,geo_jacobians))

6×10 Array{Float64,2}:
 -0.0  -0.0  -0.0  -0.0  -0.0981    -0.0         -0.0         -0.0         -0.0         -0.0       
 -0.0  -0.0  -0.0  -0.0  -0.0981    -0.0         -0.0         -0.0         -0.0         -0.0       
 -0.0  -0.0  -0.0  -0.0   0.0981    -0.0         -0.0         -0.0         -0.0         -0.0       
 -0.0  -0.0  -0.0  -0.0   0.0981    -0.0         -0.0         -0.0         -0.0         -0.0       
 -0.0  -0.0  -0.0  -0.0  -0.15625   -0.0         -0.0         -0.0         -0.0         -0.00488281
 -0.0  -0.0  -0.0  -0.0  -0.757264   0.00962361   0.00962361   0.00962361   0.00962361   0.00767049


In [41]:
z = vcat(α_vect0,c_vect0)
num_iter = 100
α_sgd = .99
for iter = 1:num_iter
    q0 = [1., 0., 0., 0., 0., 0., 0.]
    v0 = [0., 0., 0., 0., 0. , 0.]
    u0 = zeros(num_v)
    qnext = [1., 0., 0., 0., 0., 0., 0.]
    vnext = v0 + (rand(6)*2. - 1.)

    set_configuration!(x0, q0)
    set_velocity!(x0, v0)
    setdirty!(x0)
    set_configuration!(xnext, qnext)
    set_velocity!(xnext, vnext)
    setdirty!(xnext)

    H = mass_matrix(x0)
    for i = 1:num_contacts
        v = point_velocity(twist_wrt_world(xnext,bodies[i]), transform_to_root(xnext, contact_points[i].frame) * contact_points[i])
        Dtv[:,i] = map(contact_bases[i]) do d
            dot(transform_to_root(xnext, d.frame) * d, v)
        end
        rel_transforms[i] = (relative_transform(xnext, obstacles[i].contact_face.outward_normal.frame, world_frame),
                             relative_transform(xnext, contact_points[i].frame, world_frame))
        geo_jacobians[i] = geometric_jacobian(xnext, paths[i])

        ϕs[i] = RigidBodyDynamics.separation(obstacles[i], transform(xnext, contact_points[i], obstacles[i].contact_face.outward_normal.frame))
    end    
    config_derivative = configuration_derivative(xnext)
    HΔv = H * (vnext - v0)
    dyn_bias = dynamics_bias(xnext);
    
    x_ipopt = ipopt_solve(var0,f,h,g,num_h,num_g)
    
    x_lag = augmented_lagrangian_method(var0,f,h,g,num_h,num_g,z[1:num_steps],z[num_steps+1:2*num_steps],I)
    J = ForwardDiff.jacobian(z̃ -> augmented_lagrangian_method(var0,f,h,g,num_h,num_g,z̃[1:num_steps],z̃[num_steps+1:2*num_steps],I), z)
    
    loss = .5 * dot(x_ipopt - x_lag, x_ipopt - x_lag)
    z = z + α_sgd^iter*J'*(x_ipopt - x_lag)
    z = min.(1000.,max.(1., z))
    
    println(loss)
    println(z)
    println("***")
end

15.550504429837009
[1.0, 1.0, 1.0, 1.0, 1.0, 2.52111, 4.52111, 8.52111, 16.5211, 32.4691]
***
30.08727935315733
[1.0, 1.0, 1.0, 1.0, 17.9579, 2.26249, 4.26249, 8.26249, 16.2625, 32.0905]
***
0.8126424576282342
[1.0, 1.0, 1.0, 1.0, 17.9536, 2.26387, 4.26387, 8.26387, 16.2639, 32.0949]
***
6.090114097609055
[1.0, 1.0, 1.0, 1.0, 17.9434, 2.26729, 4.26729, 8.26729, 16.2673, 32.1032]
***
1.3864132190269454
[1.0, 1.0, 1.0, 1.0, 17.9451, 2.26692, 4.26692, 8.26692, 16.2669, 32.1037]
***
0.9771962287762985
[1.0, 1.0, 1.0, 1.0, 17.9398, 2.26861, 4.26861, 8.26861, 16.2686, 32.1082]
***
5.796730076119141
[1.0, 1.0, 1.0, 1.0, 17.9309, 2.27155, 4.27155, 8.27155, 16.2715, 32.1159]
***
73.9816454735046
[1.0, 1.0, 1.0, 1.0, 18.0846, 2.22939, 4.22939, 8.22939, 16.2294, 32.0781]
***
12.749527801035546
[1.0, 1.0, 1.0, 1.0, 18.1266, 2.21785, 4.21785, 8.21785, 16.2178, 32.0673]
***
104.4508242733356
[1.0, 1.0, 1.0, 1.0, 18.3678, 2.14992, 4.14992, 8.14992, 16.1499, 32.0034]
***
15.897798293292034
[1.0, 1.0, 